In [24]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [25]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-3 cm-cr-review-stars-spacing-big'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_productdescription(soup):
    try:
        product_description = soup.find("div", attrs={'id':'productDescription'})
        product_description = product_description.find("span").string.strip()

    except AttributeError:
        product_description = "Not Available"	

    return product_description

def get_manufacturer(soup):
    try:
        manufacturer= soup.find("th", attrs={'class':'a-color-secondary a-size-base prodDetSectionEntry'})
        manufacturer = manufacturer.find("th").string.strip()
        #manufacturer_value= manufacturer.text
        #manufacturer_srting= manufacturer_value.strip()       
        

    except AttributeError:
        manufacturer = "Not Available"	

    return manufacturer

def get_ansi(soup):
    try:
        ansi_element = soup.find("th", attrs={'class':'a-color-secondary a-size-base prodDetSectionEntry'})
        ansi_element = ansi_element.find("th").string.strip()

    except AttributeError:
        ansi_element = "Not Available"	
    
    return ansi_element

In [26]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[], "product_description":[], "manufacturer":[], "ansi_element":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in/" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['product_description'].append(get_productdescription(new_soup))
        d['manufacturer'].append(get_manufacturer(new_soup))
        d['ansi_element'].append(get_ansi(new_soup))
        

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [27]:
amazon_df

,title,price,rating,reviews,product_description,manufacturer,ansi_element
0,"Zebronics Camp1, 14 Liters, 1 Compartment Lapt...",₹528.00,3.0 out of 5 stars,6 ratings,"Zebronics Camp1, 14 Liters, 1 Compartment Lapt...",Not Available,Not Available
1,"ZEBRONICS Techshield A3, 23 Liters 2 Compartme...",₹846.00,5.0 out of 5 stars,6 ratings,"ZEBRONICS Techshield A3, 23 Liters 2 Compartme...",Not Available,Not Available
2,Skybags Brat Black 46 Cms Casual Backpack,₹672,4.1 out of 5 stars,"5,525 ratings",Ideal for a college student who does not carry...,Not Available,Not Available
3,American Tourister Fizz Large Size 32 Ltrs Cas...,"₹1,199.00",4.0 out of 5 stars,"55,249 ratings",Not Available,Not Available,Not Available
4,Half Moon Large 37L Laptop Bag Backpack for me...,₹849,3.9 out of 5 stars,"3,442 ratings","This backpack is lightweight water-resistant, ...",Not Available,Not Available
6,Wesley Milestone 2.0 Casual Waterproof Laptop ...,₹588.00,4.3 out of 5 stars,"12,344 ratings",Not Available,Not Available,Not Available
7,FUR JADEN Anti Theft Number Lock Backpack Bag ...,₹679.00,4.0 out of 5 stars,"5,611 ratings",Stay organized all day with a durable and ligh...,Not Available,Not Available
8,TRUE HUMAN EMPEROR® Anti-Theft backpack With U...,₹599.00,3.7 out of 5 stars,"2,462 ratings",Not Available,Not Available,Not Available
9,Half Moon 35 Ltrs Water Resistant 15.6 inch La...,₹649,3.9 out of 5 stars,"16,960 ratings",Not Available,Not Available,Not Available
10,Half Moon Large 37L Laptop Bag Backpack for me...,₹849.00,3.9 out of 5 stars,"3,442 ratings","This backpack is lightweight water-resistant, ...",Not Available,Not Available
